In [1]:
import sys
sys.path[0] = '/tensorflow-2.1.0/python3.6'
from google.colab import drive
drive.mount('/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /drive


In [0]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, Sequential, losses, optimizers, datasets
from tensorflow.keras.utils import to_categorical

In [3]:
tf.__version__

'2.1.0'

In [4]:
(x, y), (x_test, y_test) = datasets.cifar100.load_data()
print(x.shape,y.shape,x_test.shape,y_test.shape)
print(x.dtype,y.dtype)

169009152/169001437 [==============================] - 2s 0us/step
(50000, 32, 32, 3) (50000, 1) (10000, 32, 32, 3) (10000, 1)
uint8 int64


In [0]:
def preprocess(x, y):
    x = tf.cast(x, dtype=tf.float32) / 255.
    y = tf.cast(to_categorical(tf.squeeze(tf.cast(y, dtype=tf.int32), axis=1), num_classes=100), dtype=tf.int32)
    return x,y

In [0]:
class BasicBlock(layers.Layer):
    
    def __init__(self, filter_num, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = layers.Conv2D(filter_num, (3, 3), strides=stride, padding='same')
        self.bn1 = layers.BatchNormalization()
        self.relu = layers.Activation('relu')
        self.drop1 = layers.Dropout(0.5)
        self.conv2 = layers.Conv2D(filter_num, (3, 3), strides=1, padding='same')
        self.bn2 = layers.BatchNormalization()
        self.drop2 = layers.Dropout(0.5)
        if stride != 1:
            self.downsample = layers.Conv2D(filter_num, (1, 1), strides=(stride, stride), padding='valid')
        else:
            self.downsample = lambda x:x
    
    def call(self, inputs, training=None):
        out = self.conv1(inputs)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.drop1(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out = self.drop2(out)

        identity = self.downsample(inputs)
        output = layers.add([identity, out])
        output = tf.nn.relu(output)
        return output

In [0]:
class ResNet(keras.Model):

    def __init__(self, layer_dims, num_classes=100):
        super(ResNet, self).__init__()
        self.stem = Sequential([layers.Conv2D(64, (3, 3), strides=(1, 1), padding='same'),
                                layers.BatchNormalization(),
                                layers.Activation('relu'),
                                layers.MaxPool2D(pool_size=(2, 2), strides=(1, 1), padding='same')
                               ])
        self.block1 = self.build_resblock(64, layer_dims[0])
        self.block2 = self.build_resblock(128, layer_dims[1], stride=2)
        self.block3 = self.build_resblock(256, layer_dims[2], stride=2)
        self.block4 = self.build_resblock(512, layer_dims[3], stride=2)
        self.avgpool = layers.GlobalAveragePooling2D()
        self.fc = layers.Dense(num_classes, activation=tf.nn.softmax)
    
    def call(self, inputs, training=None):
        x = self.stem(inputs)
        x = self.block1(x)
        x = self.block2(x)       
        x = self.block3(x)     
        x = self.block4(x)
        x = self.avgpool(x)
        x = self.fc(x)
        return x
      
    def build_resblock(self, filter_num, blocks, stride=1):
      res_blocks = Sequential()
      res_blocks.add(BasicBlock(filter_num, stride))
      for _ in range(1, blocks):
          res_blocks.add(BasicBlock(filter_num, stride=1))
      return res_blocks

In [0]:
def main(x, y, x_test, y_test):
    epochs = 100
    model = ResNet([2, 2, 2, 2])
    model.build(input_shape=(None, 32, 32, 3))
    model.summary()
    save_best = keras.callbacks.ModelCheckpoint('/drive/My Drive/Github/CNN/ResNet_best_model.h5', monitor='val_loss', verbose=1, save_best_only=True, mode='min')
    early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', verbose=1, min_delta=0, patience=100, mode='auto')
    callbacks_list = [early_stop, save_best]
    model.compile(optimizer=optimizers.Adam(lr=1e-2),
                 loss=losses.categorical_crossentropy,
                 metrics=['accuracy'])
    x, y = preprocess(x, y)
    x_test, y_test = preprocess(x_test, y_test)
    history = model.fit(x=x, y=y, epochs=epochs, batch_size=512, validation_data=(x_test, y_test), verbose=1, callbacks=callbacks_list)
    return history

In [138]:
main(x, y, x_test, y_test)

Model: "res_net_35"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential_184 (Sequential)  multiple                  2048      
_________________________________________________________________
sequential_185 (Sequential)  multiple                  148736    
_________________________________________________________________
sequential_186 (Sequential)  multiple                  526976    
_________________________________________________________________
sequential_187 (Sequential)  multiple                  2102528   
_________________________________________________________________
sequential_188 (Sequential)  multiple                  8399360   
_________________________________________________________________
global_average_pooling2d_32  multiple                  0         
_________________________________________________________________
dense_32 (Dense)             multiple                  5

KeyboardInterrupt: ignored